In [ ]:
%tensorflow_version 1.x  # tf version 2 does not work with BERT

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x  # tf version 2 does not work with BERT`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


#Installing and Testing BERT-as-Service

In [ ]:
!pip install bert-serving-client
!pip install -U bert-serving-server[http]

     |████████████████████████████████| 71kB 5.1MB/s 
     |████████████████████████████████| 358kB 17.6MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=980640c772a75d6ce939f345efd7f58e863e1752bfbf764c669974ef26ebb85a
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil


In [ ]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip
!nohup bert-serving-start -model_dir=./uncased_L-12_H-768_A-12 > out.file 2>&1 &

--2021-05-30 13:51:04--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.202.128, 74.125.20.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M  96.4MB/s    in 4.0s    

2021-05-30 13:51:08 (96.4 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [ ]:
!ls

out.file     tmpDVkxrW	tmpMgjwnP  tmpTE3KZ8  uncased_L-12_H-768_A-12
sample_data  tmpHobzbu	tmpPLcwjI  tmpu02F3f  uncased_L-12_H-768_A-12.zip
tmp9Vwzv3    tmpiscxfB	tmpPYe3fj  tmpVP4B7m


In [ ]:
from bert_serving.client import BertClient

In [ ]:
bc = BertClient()

In [ ]:
list_text = ['Running BERT as service']
embedded_text = bc.encode(list_text)

In [ ]:
embedded_text.shape

(1, 768)

In [ ]:

print(embedded_text)

[[-7.11947903e-02 -1.68446779e-01  1.94467809e-02 -1.18920840e-01
   2.24907279e-01  6.40937984e-02  5.11571348e-01  8.90361741e-02
  -2.38681078e-01  1.17627703e-01  2.26348042e-01  1.48783669e-01
   1.87713802e-01  2.62925744e-01 -4.88718182e-01 -3.90095711e-02
  -4.39551264e-01  5.88086247e-01  5.68010658e-02 -1.32986888e-01
  -3.11084986e-01  1.13405682e-01  2.02498093e-01 -7.46200860e-01
  -2.38194987e-01  1.11216597e-01 -4.87747975e-02  2.08614338e-02
  -4.20628697e-01 -4.81931679e-02  2.28437126e-01  1.50565609e-01
   3.44260663e-01  1.78000867e-01 -3.96239966e-01 -7.00553179e-01
   4.80400562e-01  2.80852318e-01 -3.82272631e-01  2.02066839e-01
  -2.28433847e-01  8.05517733e-02  8.01060259e-01 -2.99840212e-01
  -7.52732456e-02 -9.66721475e-02 -3.14535290e-01 -3.39455932e-01
   4.29213881e-01 -5.82820356e-01 -1.96047425e-01 -1.52280167e-01
  -3.07475775e-01  2.66136918e-02  2.47104049e-01  1.08324401e-01
   7.98351109e-01 -5.83960831e-01  3.85459810e-01  8.29728097e-02
   1.02527

# Using BERT for Text Classification Problem

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

# load training data
train = pd.read_csv('/content/drive/My Drive/Data/BERT_proj/train_E6oV3lV.csv', encoding='iso-8859-1')
train.shape

(31962, 3)

In [ ]:
train.head(2)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...


In [ ]:
import re

# clean text from noise
def clean_text(text):
    # filter to allow only alphabets
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    # remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    
    # convert to lowercase to maintain consistency
    text = text.lower()
       
    return text

In [ ]:
train['clean_text'] = train.tweet.apply(clean_text)

In [ ]:
from sklearn.model_selection import train_test_split

# split into training and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(train.clean_text, train.label, test_size=0.25, random_state=42)

print('X_tr shape:',X_tr.shape)

X_tr shape: (23971,)


In [ ]:
# get the embedding for train and val sets
X_tr_bert = bc.encode(X_tr.tolist())
X_val_bert = bc.encode(X_val.tolist())

/usr/local/lib/python3.7/dist-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [ ]:
X_tr_bert.shape

(23971, 768)

In [ ]:
from sklearn.linear_model import LogisticRegression

# LR model
model_bert = LogisticRegression()
# train
model_bert = model_bert.fit(X_tr_bert, y_tr)
# predict
pred_bert = model_bert.predict(X_val_bert)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_val, pred_bert))

0.9508196721311475


Reference:

https://github.com/hanxiao/bert-as-service/issues/380